# Machin learning(ML) y Dinámica Estructural

```
Autor       : Francisco Jimenez Mucho
Linkedin    : https://www.linkedin.com/in/fjmucho/
date:       : 7 julio 2025
```

## Solving `Poison equation in 1D` Using Physics Informed Neural Networks (PINNs)

### Example 01.

Considere el siguente ejemplo boundary value problem (BVP) 'Posison equation in 1D' ref.

$$
\dfrac{d^2 y}{d x^2} + \pi^2\sin{(\pi x)} = 0,\,\,\,\, x\in [-1,1]
$$

Condiciones de borde

$$
y(-1)=0 \,\, and \,\, y(1)=0
$$

**Solucion Análitica**

$$y(x) = \sin{(\pi x)} $$

### Algoritmo

1. **Definir la red Neuronal | Define Neural Network**
    - **Input**: Spatial Coordinate $x$.
    - **Hidden Layers**: Non-linear transformations to approximate the solution
    - **outputs**: Predited $y_{NN}(x)$
2. **Compute Derivatives**
    - Use `tf.GradientTape` to compute $\frac{d^2y}{dx^2}$
3. **Define Residual**
    - Compute the residual of the differential equation.
    $$residual = \frac{d^2y(x)}{dx^2} + \pi^2\sin{(\pi x)}$$
4. **Loss Function**
    - Combine residual loss and boundary loss:
    $$loss=MSE(resudual) + MSE(boundary conditions)
5. **Training**
    - Minimize the loss function using an optimizer (e.g. Adam)
6. **Evaluate**
    - Predict and plot the output $y$ over the domain compare with the analytical solution.
    

### 1 <img alt="pagakes" src="https://pypi.org/static/images/white-cube.2351a86c.svg" width="1%"/> Importacion de Librerias, clases, funciones, constantes, etc.

In [1]:
import numpy as np 

In [2]:
from matplotlib import pyplot as plt
from matplotlib import cm

In [3]:
import tensorflow as tf

In [4]:
plt.close('all')
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Times New Roman'
plt.rcParams['font.size'] = 10
plt.rcParams['figure.dpi'] = 100

#### 1a. Funciones Utiles | Initial setup

In [5]:
def create_model():
    model = {
        'dense1': tf.keras.layers.Dense(50, activation='tanh', input_shape=(1,)),
        'dense2': tf.keras.layers.Dense(50, activation='tanh'),
        'dense3': tf.keras.layers.Dense(50, activation='tanh'),
        'output_layer': tf.keras.layers.Dense(1)
    }
    return model

def call_model(model, x):
    x = model['dense1'](x)
    x = model['dense2'](x)
    x = model['dense3'](x)
    return model['output_layer'](x)

# define  the differential equation using tf.GradientTape
def differential_equation_PDE(x, model):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x)
        u_pred = call_model(model, x)
        u_x = tape.gradient(u_pred, x)
    u_xx = tape.gradient(u_x, x)
    del tape  # free memory

    return u_xx + np.pi**2 * tf.sin(np.pi * x) # define the loss function

def loss_function(model, x, x_bc, y_bc):
    res = differential_equation_PDE(x, model)
    # compute the main square error of the boundary conditions
    loss_PDE = tf.reduce_mean(tf.square(res))
    y_bc_pred = call_model(model, x_bc)
    # compute the square error of the boundary conditions
    loss_bc = tf.reduce_mean(tf.square(y_bc - y_bc_pred))
    return loss_PDE + loss_bc

def train_step(model, x, x_bc, y_bc, optimizer):
    with tf.GradientTape() as tape:
        loss_value = loss_function(model, x, x_bc, y_bc)
    grads = tape.gradient(loss_value, [layer.trainable_variables for layer in model.values()])
    grads = [item for sublist in grads for item in sublist]  # flatten the list of gradients
    variables = [var for layer in model.values() for var in layer.trainable_variables]
    optimizer.apply_gradients(zip(grads, variables))

    return loss_value

### 2. Carga y preparacion de datos (data)

## Referencias.

1. 
<!-- 5. Elastropy videos: https://youtu.be/pq3aAWU6kBQ, https://www.youtube.com/playlist?list=PLM7DTyYjZGuLYDLV9g7AqkEatySe-lAqG -->